In [1]:
import torch
import numpy as np
from tqdm import tqdm
from transformers import pipeline, BertForSequenceClassification, AutoTokenizer
from datasets import Features, Value
from tqdm import tqdm
from huggingface_hub import notebook_login
#git config --global credential.helper store
#https://github.com/git-ecosystem/git-credential-manager/releases/tag/v2.5.1

In [5]:
notebook_login()

In [6]:
tokenizer=AutoTokenizer.from_pretrained('bert-base-uncased')

In [7]:
from datasets import load_dataset
from datasets import Dataset
from tqdm import tqdm
dataset=load_dataset("SetFit/enron_spam")

Repo card metadata block was not found. Setting CardData to empty.


In [8]:
labels=['ham', 'spam']
id2label = {
    0: "ham",
    1: "spam"
}
label2id = {
    "ham": 0,
    "spam": 1
}
print (labels)

In [9]:
dataset['train']=dataset['train'].shuffle().select(range(5000))
dataset['train']

Dataset({
    features: ['message_id', 'text', 'label', 'label_text', 'subject', 'message', 'date'],
    num_rows: 5000
})

In [10]:
dataset['test']=dataset['test'].shuffle().select(range(500))
dataset['test']

Dataset({
    features: ['message_id', 'text', 'label', 'label_text', 'subject', 'message', 'date'],
    num_rows: 500
})

In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['message_id', 'text', 'label', 'label_text', 'subject', 'message', 'date'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['message_id', 'text', 'label', 'label_text', 'subject', 'message', 'date'],
        num_rows: 500
    })
})

In [12]:
#[0, 1] - Spam
#[1, 0] - Ham
def preprocess_data(examples):
    text = examples["text"]
    encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
    encoding["labels"] = [[0.0, 1.0] if label==1 else [1.0, 0.0] for label in examples['label']]
    return encoding

encoded_dataset=dataset.map(preprocess_data, remove_columns=dataset['train'].column_names, batched=True)
encoded_dataset #TODO: Batching

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 500
    })
})

In [13]:
print (encoded_dataset['train']['labels'][:20])

In [14]:
example = encoded_dataset['train'][0]
print(example.keys())
tokenizer.decode(example['input_ids'])

'[CLS] joint venture some emerging documents ( early days ) but good for information purposes - - - - - - - - - - - - - - - - - - - - - - forwarded by mike jordan / lon / ect on 06 / 04 / 2001 14 : 42 - - - - - - - - - - - - - - - - - - - - - - - - - - - coralie evans 05 / 04 / 2001 08 : 52 to : mike jordan / lon / ect @ ect cc : subject : joint venture please find attached the draft soap ( pretty rough ) and the business [SEP]'

In [15]:
encoded_dataset.set_format("torch")

In [16]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id
                                                     )

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
batch_size = 30
metric_name = "f1"

In [18]:
model_path=f"bert-mail-spam-finetuned"

In [19]:
from transformers import TrainingArguments, Trainer
args = TrainingArguments(
    model_path,
    eval_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #label_names=labels,
    #push_to_hub=True,
)

In [20]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [21]:
encoded_dataset['train'][0]['labels'].type()

'torch.FloatTensor'

In [22]:
encoded_dataset['train']['input_ids'][0]

tensor([  101,  4101,  6957,  2070,  8361,  5491,  1006,  2220,  2420,  1007,
         2021,  2204,  2005,  2592,  5682,  1011,  1011,  1011,  1011,  1011,
         1011,  1011,  1011,  1011,  1011,  1011,  1011,  1011,  1011,  1011,
         1011,  1011,  1011,  1011,  1011,  1011,  1011,  2830,  2098,  2011,
         3505,  5207,  1013,  8840,  2078,  1013, 14925,  2102,  2006,  5757,
         1013,  5840,  1013,  2541,  2403,  1024,  4413,  1011,  1011,  1011,
         1011,  1011,  1011,  1011,  1011,  1011,  1011,  1011,  1011,  1011,
         1011,  1011,  1011,  1011,  1011,  1011,  1011,  1011,  1011,  1011,
         1011,  1011,  1011,  1011, 11034,  2666,  6473,  5709,  1013,  5840,
         1013,  2541,  5511,  1024,  4720,  2000,  1024,  3505,  5207,  1013,
         8840,  2078,  1013, 14925,  2102,  1030, 14925,  2102, 10507,  1024,
         3395,  1024,  4101,  6957,  3531,  2424,  4987,  1996,  4433,  7815,
         1006,  3492,  5931,  1007,  1998,  1996,  2449,   102])

In [23]:
#forward pass
outputs = model(
    input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), 
    labels=encoded_dataset['train'][0]['labels'].unsqueeze(0)
)
outputs

SequenceClassifierOutput(loss=tensor(0.6370, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[ 0.1364, -0.0950]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [24]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [25]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,No log,0.055938,0.982036,0.982000,0.980000
2,No log,0.034712,0.994000,0.994000,0.994000
3,0.083300,0.044948,0.988000,0.988000,0.988000
4,0.083300,0.054111,0.988000,0.988000,0.988000
5,0.083300,0.057024,0.988000,0.988000,0.988000


TrainOutput(global_step=835, training_loss=0.0515995609546136, metrics={'train_runtime': 506.301, 'train_samples_per_second': 49.378, 'train_steps_per_second': 1.649, 'total_flos': 1644444096000000.0, 'train_loss': 0.0515995609546136, 'epoch': 5.0})

In [26]:
trainer.evaluate()

{'eval_loss': 0.03471248224377632,
 'eval_f1': 0.994,
 'eval_roc_auc': 0.994,
 'eval_accuracy': 0.994,
 'eval_runtime': 3.2806,
 'eval_samples_per_second': 152.41,
 'eval_steps_per_second': 5.182,
 'epoch': 5.0}

In [28]:
trainer.save_model(model_path)

In [29]:
trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/alexsd06/bert-mail-spam-finetuned/commit/e6f0874e45551c741e183cee042e89dbfc334cb7', commit_message='End of training', commit_description='', oid='e6f0874e45551c741e183cee042e89dbfc334cb7', pr_url=None, pr_revision=None, pr_num=None)

In [30]:
pipe = pipeline('text-classification', model=model_path, device='cuda', truncation=True)

In [31]:
message="""
The interview is in 2 days! Best regards, David!
"""
pipe(message)

[{'label': 'ham', 'score': 0.9937520623207092}]

In [33]:
idx='test'
l=len(dataset[idx])
good, bad = 0, 0
shown=0
for item in tqdm(dataset[idx]):
    text=item['text']
    label=item['label']
    label=id2label[label]
    predict=pipe(text)[0]['label']
    #if predict=='spam':
    #    print ('spam') #The model nevers predicts spam!!!
    #    break
    if predict==label:
        good+=1
    else:
        bad+=1

100%|█████████████████████████████████████████| 500/500 [00:07<00:00, 69.29it/s]


In [40]:
print (f'Accuracy {100*good/l}%!')
print (f'Loss {100*bad/l}%!')

**Raport:**
1. Training data: Accuracy 99.56%! Loss 0.44%!
2. Test data: Accuracy 99.4%! Loss 0.6%!

**Note:** This was done on a small sample of the dataset: 5000 training, 500 testing